In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer


df = pd.read_csv('data/labelled/labelled_data.csv')

df = df.dropna(subset=['label'])

# Lista para armazenar os dados
data = []
labels = []
added_labels = set()

batch_size = 32
img_height = 180
img_width = 180
data_dir='data/cuted/'

2024-03-26 16:28:00.428386: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 16:28:00.429754: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 16:28:00.508511: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 16:28:00.826868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 16:28:01.937705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 602 files belonging to 6 classes.


Using 482 files for training.


2024-03-26 16:28:04.560035: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 16:28:04.560693: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Pré-processamento de imagens: 
Você pode adicionar mais etapas de pré-processamento de imagens, como normalização, aumentação de dados, etc.

## Paralelização: 
A leitura e o processamento de imagens podem ser paralelizados usando multiprocessing ou joblib para acelerar o processo.

## Ajuste de hiperparâmetros: 
Você pode usar técnicas como GridSearchCV ou RandomizedSearchCV para encontrar os melhores hiperparâmetros para seus modelos.

## Validação cruzada: 
Em vez de dividir os dados apenas uma vez, você pode usar a validação cruzada para obter uma estimativa mais robusta do desempenho do modelo.

## Verificação de overfitting: 
Verifique se o modelo está overfitting nos dados de treinamento. Se estiver, você pode tentar adicionar regularização ou usar um modelo mais simples.

In [3]:
# class ProcessImage:
#     def __init__(self, df):
#         self.df = df

#     def process_single_image(self, row):
#         img = Image.open(row['image_path'])
#         img = img.resize((260, 260))
#         img_array = np.array(img).flatten()
#         return img_array, row['label']

#     def process_images(self):
#         results = Parallel(n_jobs=-1)(delayed(self.process_single_image)(row) for _, row in self.df.iterrows())
#         data, labels = zip(*results)
#         return data, labels


# class TrainModel:
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels

#     def train(self):
#         X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#         # Criar e treinar os modelos com validação cruzada e normalização
#         mlp = MLPClassifier(random_state=42)
#         rf = RandomForestClassifier(random_state=42)

#         print("MLP Cross-Validation Score:")
#         print(np.mean(cross_val_score(mlp, X_train, y_train, cv=5)))

#         print("RandomForest Cross-Validation Score:")
#         print(np.mean(cross_val_score(rf, X_train, y_train, cv=5)))

In [4]:
# Iterar sobre cada linha do DataFrame
for index, row in df.iterrows():
    # Abrir a imagem
    img = Image.open(row['image_path'])
    img = img.resize((160, 160))
    
    # Converter a imagem em um array numpy e achatá-la
    img_array = np.array(img).flatten()
    
    img_array = img_array / 255.0
    
    # Adicionar o array de imagem e o rótulo à lista de dados
    data.append(img_array)
    labels.append(row['label'])
    
# Crie um binarizador de labels
lb = LabelBinarizer()

# Ajuste o binarizador de labels e transforme as labels
labels = lb.fit_transform(labels)

scaler = StandardScaler()


# # X_train_scaled = scaler.fit_transform(X_train)
# # X_test_scaled = scaler.transform(X_test)

# # # Redução de dimensionalidade
# # pca = PCA(n_components=0.95)
# # X_train_pca = pca.fit_transform(X_train_scaled)
# # X_test_pca = pca.transform(X_test_scaled)

# # Treinar um modelo MLP com paralelização e ajuste de hiperparâmetros
# mlp = MLPClassifier(random_state=42, activation='relu', learning_rate='adaptive', n_jobs=-1)
# mlp.fit(X_train_pca, y_train)
# mlp_predictions = mlp.predict(X_test_pca)
# mlp_accuracy = accuracy_score(y_test, mlp_predictions)
# print(f"MLP Accuracy: {mlp_accuracy}")



In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)